# 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助您：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(agent101)`，即可执行


In [ ]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate agent101

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "agent101" ]; then
    echo "✅ 当前单元格已成功激活到 agent101 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(agent101)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate agent101"
fi

echo "========================================="


In [ ]:
# 2. 设置pip 为清华源
%pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config


In [ ]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT


In [ ]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助您：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


# 第八章：避免幻觉

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [1]:
# 安装OpenAI库
%pip install openai==1.61.0

# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://vip.apiyi.com/v1
  🤖 模型: gpt-4o
  🌐 API地址: https://vip.apiyi.com/v1
  🔑 API密钥: sk-R2utG...B944

✅ 使用统一配置管理成功！



## 课程 

### 什么是LLM幻觉？

**LLM幻觉**（AI Hallucination）是指大语言模型生成看似合理但实际上不准确、虚假或无法验证的信息的现象。这是当前AI技术面临的一个重要挑战。

### 幻觉产生的原因

1. **训练数据限制**：模型基于有限的训练数据进行推理
2. **概率性生成**：模型基于概率分布生成文本，可能产生不存在的"合理"组合
3. **知识边界模糊**：模型难以准确识别自己的知识边界
4. **上下文理解偏差**：对复杂问题的理解可能存在偏差

### 避免幻觉的核心策略

**好消息是：有多种技术可以显著减少幻觉现象**。本节将重点介绍以下策略：

1. **🛡️ 承认无知策略**：给模型选择说"不知道"的机会
2. **🔍 证据导向策略**：要求模型先寻找证据再回答
3. **📝 分步推理策略**：引导模型逐步分析问题
4. **✅ 可信度评估**：要求模型评估自己答案的可信度

### 为什么避免幻觉很重要？

- **增强用户信任**：准确的信息建立用户对AI系统的信任
- **降低误导风险**：避免传播错误信息
- **提高实用性**：确保AI助手在实际应用中的可靠性
- **符合伦理要求**：负责任的AI开发实践

下面我们通过具体示例来学习这些技术的实际应用。

## 示例1：承认无知策略 🛡️

**核心思想**：明确告诉模型，如果不确定答案，可以诚实地说"不知道"，而不是编造信息。

### 问题场景
假设我们询问一个可能不存在或者模型不确定的信息。让我们比较两种提示方式的效果。


In [6]:
# 📌 错误示例：没有给模型"不知道"的选项
bad_prompt = """
请告诉我关于 林黛玉倒把垂杨柳的故事。
"""

print("❌ 容易产生幻觉的提示:")
print(bad_prompt)
print("=" * 50)

response_bad = get_completion(bad_prompt)
print("模型回答:")
print(response_bad)


❌ 容易产生幻觉的提示:

请告诉我关于 林黛玉倒把垂杨柳的故事。

模型回答:
林黛玉倒把垂杨柳的故事出自中国古典小说《红楼梦》，是其中一个非常著名的情节。林黛玉是《红楼梦》中的主要人物之一，她性格敏感、多愁善感，常常以诗词表达自己的情感。

这个故事发生在大观园的一次诗社活动中。贾宝玉、林黛玉、薛宝钗等人在园中聚会，大家决定以“柳”为题作诗。林黛玉在作诗时，写下了“留得残荷听雨声”这样的句子，表现出她对生命无常的感慨和对自然景物的细腻观察。

在这次活动中，林黛玉的诗被认为是“倒把垂杨柳”，意思是她的诗意境深远，超越了常规的思维方式，给人一种耳目一新的感觉。这个故事不仅展示了林黛玉的才华，也反映了她内心的孤独和对生活的独特理解。

林黛玉倒把垂杨柳的故事体现了《红楼梦》中人物之间的互动和情感交流，同时也展示了作者曹雪芹对诗词艺术的深刻理解和运用。


In [7]:
# ✅ 正确示例：明确给出"不知道"的选项
good_prompt = """
请告诉我关于 林黛玉倒把垂杨柳的故事梗概和故事详情。

重要提示：
- 如果你对这个故事不确定或没有可靠信息，请直接说"我不知道"或"我没有这个产品的确切信息"
- 不要编造或故事梗概和故事详情
- 只提供你确信准确的信息
"""

print("✅ 避免幻觉的提示:")
print(good_prompt)
print("=" * 50)

response_good = get_completion(good_prompt)
print("模型回答:")
print(response_good)


✅ 避免幻觉的提示:

请告诉我关于 林黛玉倒把垂杨柳的故事梗概和故事详情。

重要提示：
- 如果你对这个故事不确定或没有可靠信息，请直接说"我不知道"或"我没有这个产品的确切信息"
- 不要编造或故事梗概和故事详情
- 只提供你确信准确的信息

模型回答:
我不知道。


### 🎯 示例1总结

**对比分析**：
- **第一种提示**：容易让模型感觉"必须"给出答案，可能导致编造信息
- **第二种提示**：明确告知模型可以承认不知道，减少幻觉风险

**关键技巧**：
1. 使用"如果你不确定，请说不知道"这样的明确指示
2. 强调"不要编造或猜测"
3. 要求"只提供确信准确的信息"

---


## 示例2：证据导向策略 🔍

**核心思想**：要求模型在回答之前先分析现有信息，基于证据进行推理，而不是直接给出结论。

### 问题场景
让我们以一个历史事件查询为例，看看如何通过要求模型先寻找证据来提高答案的准确性。


In [4]:
# 📌 错误示例：直接要求答案，没有要求证据分析
direct_prompt = """
中国明朝时期是否有过与美洲的贸易往来？请给出详细说明。
"""

print("❌ 直接询问的提示:")
print(direct_prompt)
print("=" * 50)

response_direct = get_completion(direct_prompt)
print("模型回答:")
print(response_direct)


❌ 直接询问的提示:

中国明朝时期是否有过与美洲的贸易往来？请给出详细说明。

模型回答:
关于中国明朝时期与美洲的贸易往来，历史记录中并没有明确的证据表明存在直接的贸易联系。明朝（1368-1644年）是中国历史上一个重要的朝代，以其强大的海上力量和郑和下西洋等著名航海活动而闻名。然而，这些航海活动主要集中在东南亚、南亚和东非地区，并没有明确记录显示明朝与美洲有直接的贸易往来。

以下是一些相关背景信息：

1. **郑和下西洋**：郑和是明朝著名的航海家，他在1405年至1433年间进行了七次大规模的航海活动。这些航海活动主要是为了宣扬明朝的国威和进行外交交流，航线主要覆盖东南亚、南亚、中东和东非地区，并没有到达美洲。

2. **地理知识的限制**：在明朝时期，中国对美洲的地理知识非常有限。虽然有一些关于遥远土地的传说，但没有具体的航海技术或地理知识支持与美洲的直接联系。

3. **欧洲人的到来**：美洲与亚洲的直接联系主要是在欧洲人到达美洲之后才逐渐建立起来。哥伦布在1492年到达美洲后，欧洲国家开始在美洲进行殖民活动，并逐渐建立了与亚洲的贸易联系。

4. **间接影响**：虽然没有直接的贸易往来，但通过欧洲的中介，明朝时期的中国商品可能间接地进入了美洲市场。例如，中国的瓷器、丝绸等商品通过欧洲的贸易网络可能被带到美洲。

综上所述，明朝时期中国与美洲没有直接的贸易往来，任何可能的联系都是通过欧洲的中介实现的。


In [5]:
# ✅ 正确示例：要求基于证据的分步分析
evidence_based_prompt = """
关于中国明朝时期是否有过与美洲的贸易往来这个问题，请按以下步骤分析：

第一步：时间线分析
- 明朝的时间范围（哪一年到哪一年）
- 欧洲人发现美洲的时间
- 这两个时间段是否有重叠

第二步：证据收集
- 列出你知道的相关历史事实和证据
- 明确指出哪些信息你确定，哪些不确定

第三步：逻辑推理
- 基于上述证据进行分析
- 考虑当时的技术条件、地理因素、政治环境

第四步：得出结论
- 给出你的答案，并说明可信度
- 如果证据不足，请明确说明

请按照这个结构来回答，不要跳过任何步骤。
"""

print("✅ 基于证据的分步提示:")
print(evidence_based_prompt)
print("=" * 50)

response_evidence = get_completion(evidence_based_prompt)
print("模型回答:")
print(response_evidence)


✅ 基于证据的分步提示:

关于中国明朝时期是否有过与美洲的贸易往来这个问题，请按以下步骤分析：

第一步：时间线分析
- 明朝的时间范围（哪一年到哪一年）
- 欧洲人发现美洲的时间
- 这两个时间段是否有重叠

第二步：证据收集
- 列出你知道的相关历史事实和证据
- 明确指出哪些信息你确定，哪些不确定

第三步：逻辑推理
- 基于上述证据进行分析
- 考虑当时的技术条件、地理因素、政治环境

第四步：得出结论
- 给出你的答案，并说明可信度
- 如果证据不足，请明确说明

请按照这个结构来回答，不要跳过任何步骤。

模型回答:
第一步：时间线分析
- 明朝的时间范围是1368年到1644年。
- 欧洲人发现美洲的时间是1492年，哥伦布首次到达美洲。
- 这两个时间段有重叠，明朝的后期与美洲被发现的时间重合。

第二步：证据收集
- 确定的信息：
  1. 明朝时期，中国的海上贸易相对活跃，郑和下西洋是一个著名的例子，但他的航行主要集中在印度洋和东非一带。
  2. 欧洲人在15世纪末开始探索和殖民美洲，西班牙和葡萄牙是最早的殖民者。
  3. 明朝实行海禁政策，限制对外贸易，尤其是在16世纪中期以后。
- 不确定的信息：
  1. 是否有直接证据表明明朝与美洲有直接贸易往来。
  2. 是否有考古或文献证据支持明朝与美洲之间的联系。

第三步：逻辑推理
- 技术条件：明朝的航海技术在当时是先进的，但主要集中在亚洲和非洲的航线。跨太平洋航行的技术和动力可能不足。
- 地理因素：太平洋的广阔和未知可能对明朝的航海活动构成障碍。
- 政治环境：明朝的海禁政策限制了对外贸易和航海活动，减少了与远距离地区的直接接触可能性。

第四步：得出结论
- 结论：基于现有的证据和逻辑推理，明朝与美洲之间没有直接的贸易往来。虽然时间上有重叠，但技术、地理和政治因素使得这种联系不太可能。
- 可信度：中等。虽然没有直接证据支持明朝与美洲的贸易，但也没有明确的证据完全排除这种可能性。由于证据不足，无法得出绝对结论。


### 🎯 示例2总结

**对比分析**：
- **直接询问**：模型可能直接给出结论，缺乏推理过程的透明度
- **证据导向**：通过分步骤要求，让模型展示推理过程，更容易发现潜在错误

**证据导向策略的关键要素**：
1. **时间线分析**：建立时间框架，避免时代错乱
2. **证据收集**：要求模型明确列出已知事实
3. **不确定性标注**：让模型区分确定和不确定的信息
4. **逻辑推理**：基于证据进行分析，而非猜测
5. **可信度评估**：要求模型评估自己答案的可靠性

**适用场景**：
- 历史事件查询
- 科学问题分析
- 复杂决策支持
- 事实核查任务

---


## 🚀 综合最佳实践

### 避免幻觉的提示工程模板

以下是一个可复用的提示模板，结合了多种避免幻觉的技术：

```
任务：[描述你的具体任务]

分析要求：
1. 首先评估你对这个问题的知识确定性
2. 如果不确定或缺乏可靠信息，请明确说明
3. 列出你将基于的事实和证据
4. 进行逐步推理分析
5. 给出结论并评估可信度（1-10分）

重要提示：
- 诚实承认知识边界，不要编造信息
- 区分事实、推测和个人观点
- 如果需要最新信息，请说明你的知识截止时间
```

### 🛠️ 实用技巧总结

| 策略 | 关键技术 | 适用场景 |
|------|----------|----------|
| **承认无知** | "如果不确定请说不知道" | 事实查询、技术规格 |
| **证据导向** | 分步推理、证据列举 | 复杂分析、历史研究 |
| **可信度评估** | 要求1-10分评分 | 决策支持、风险评估 |
| **知识边界** | 明确知识截止时间 | 时效性信息查询 |
| **多角度验证** | 要求从不同角度分析 | 争议性话题、评价 |

### ⚠️ 常见陷阱

1. **过度自信表述**：避免让模型使用过于绝对的语言
2. **忽略时效性**：某些信息可能已过时
3. **文化偏见**：注意不同文化背景下的理解差异
4. **技术局限**：模型可能在某些专业领域知识有限

### 📊 检验方法

**如何判断回答质量**：
- ✅ 模型是否承认了不确定性？
- ✅ 是否提供了推理过程？
- ✅ 事实陈述是否可验证？
- ✅ 结论是否合理谨慎？

记住：**一个承认局限性的准确答案，远比一个看似完美的错误答案更有价值！**
